# Introduction

#### Business Problem

Watering holes for great Beer often cluster together in certain areas forming up enclaves. For beer fanatics new to town it takes quite a bit of research to plan a great night out for crawling pubs as no matter how Epic the night is, often the are trade offs to be made in choosing which places to visit. By finding clusters, we could maximize Beer satisfaction for beer lovers as time spent on drunkely commuting would be reduced.
  

#### Audience

Other than Beer lovers - From a business oriented view many other audience could benefit from the same approach - tour guides/ airbnb experienece providers / hosts often would need to route plan and know which areas could maximize their client's satisfaction with the greatest convenience. Foursquare location data could help us reveal these cluster through mining the review and position data.
   
   
We would be using Singapore as an sample for the above approach  

# Data Section

In order to complete the aboce we would need the following

- Foursquare API - Explore Endpoint for list of venues
- Foursquare API - Venues Endpoint for details
- Google Maps to find inital lat/log coordinates

In [104]:
import requests
import pandas as pd
import numpy as np

In [88]:

categoryId="4d4b7105d754a06376d81259"
latnum="1.3521"
longnum="103.8198"
radiusnum="50000"
offsetnum=0

In [89]:
def pullExplorerdata(clientid,clientsecret,latnum,longnum,radiusnum,categoryId,offsetnum):
    explorerUrl=F"https://api.foursquare.com/v2/venues/explore?&client_id={clientid}&client_secret={clientsecret}&ll={latnum},{longnum}&radius={radiusnum}&categoryId={categoryId}&v=20200815&limit=50&offset={offsetnum}"
    response = requests.get(explorerUrl)
    return response.json()

In [90]:
dat=pullExplorerdata(clientid,clientsecret,latnum,longnum,radiusnum,categoryId,offsetnum)
results=[]
items=dat["response"]["groups"][0]['items']
results+=items

while dat['meta']['code']==200 and len(items)>0:
    
    offsetnum+=50
    print(offsetnum)
    dat=pulldata(clientid,clientsecret,latnum,longnum,radiusnum,categoryId,offsetnum)
    items=dat["response"]["groups"][0]['items']
    results+=items


50
100
150
200


In [91]:
df=pd.DataFrame()

In [92]:
df['id']=[i['venue']['id'] for i in results]
df['name']=[i['venue']['name'] for i in results]
df['location_lat']=[i['venue']['location']['lat'] for i in results]
df['location_lng']=[i['venue']['location']['lng'] for i in results]
df['location_post']=[i['venue']['location']['postalCode']  if 
                        'postalCode' in i['venue']['location'].keys() else np.nan for i in results]
df['location_address']=[i['venue']['location']['address']  if 
                        'address' in i['venue']['location'].keys() else np.nan for i in results]
df['location_neighborhood']=[i['venue']['location']['neighborhood']  if 
                        'neighborhood' in i['venue']['location'].keys() else np.nan for i in results]
df['location_city']=[i['venue']['location']['city']  if 
                        'city' in i['venue']['location'].keys() else np.nan for i in results]
df['location_country']=[i['venue']['location']['country']  if 
                        'country' in i['venue']['location'].keys() else np.nan for i in results]
df['cat_name']=[i['venue']['categories'][0]['name'] for i in results]
df['catid']=[i['venue']['categories'][0]['id'] for i in results]

In [94]:
def pullVenueData(venueId):
    venueUrl=F"https://api.foursquare.com/v2/venues/{venueid}?&client_id={clientid}&client_secret={clientsecret}&v=20200815"
    response = requests.get(venueUrl)
    return response.json()

In [102]:
df["fulljson"]=df["id"].apply(pullVenueData)

In [105]:
df

,id,name,location_lat,location_lng,location_post,location_address,location_neighborhood,location_city,location_country,cat_name,catid,fulljson
0,576a9b71498ed1ba6321e4ff,The Other Room,1.305297,103.832860,238865,"Level 1, Marriott Tangs",NaN,Singapore,Singapore,Speakeasy,4bf58dd8d48988d1d4941735,"{'meta': {'code': 200, 'requestId': '5f394b661..."
1,535083f7498e3044869bb48f,Manhattan,1.304911,103.825368,249715,2/F Regent Singapore,NaN,Singapore,Singapore,Cocktail Bar,4bf58dd8d48988d11e941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
2,585a1ef68ee5606a98d90f21,ATLAS Bar,1.299965,103.857768,188778,600 North Bridge Road,Downtown Core,Singapore,Singapore,Cocktail Bar,4bf58dd8d48988d11e941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
3,5bebc7963af98823a52cc62e,Whiskey Library @ The Vagabond Club,1.306981,103.858309,207630,39 Syed Alwi Road,NaN,Singapore,Singapore,Whisky Bar,4bf58dd8d48988d122941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
4,51b49e49abd88dd0b4e7330f,Napoleon Food & Wine Bar,1.279925,103.847333,068641,206 Telok Ayer Street,Tanjong Pagar,Singapore,Singapore,Wine Bar,4bf58dd8d48988d123941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
...,...,...,...,...,...,...,...,...,...,...,...,...
174,4cebaa9ce888f04d7772536b,Pasar Malam Taman Manis,1.681213,103.523741,NaN,NaN,NaN,NaN,Malaysia,Brewery,50327c8591d4c4b30a586d5d,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
175,4d2877026e27a14340371a24,Show Box,1.483623,103.390808,NaN,NaN,NaN,Pontian Kechil,Malaysia,Karaoke Bar,4bf58dd8d48988d120941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
176,4e6db6a9b9932d7f639f8329,WAN LOONG 萬隆酒店,1.479474,103.388832,NaN,NaN,NaN,NaN,Malaysia,Speakeasy,4bf58dd8d48988d1d4941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."
177,4e2fc79c1f6ed9df90234eb8,Showbox Karaoke,1.486391,103.390710,NaN,"pontian,johor",NaN,Pontian Kechil,Malaysia,Karaoke Bar,4bf58dd8d48988d120941735,"{'meta': {'code': 429, 'errorType': 'quota_exc..."


In [144]:
import folium
m = folium.Map(location=[latnum, longnum], zoom_start=11.5)


In [139]:
latlong=[list(a) for a in zip(df["location_lat"],df["location_lng"]) ]

In [142]:
for i in latlong:

    folium.Marker(
        location=i
    ).add_to(m)

In [143]:
m